In [85]:
import psycopg2
import pandas as pd
import sqlalchemy

In [86]:
try:
    pwd = "SpiderMonkey6."
    conn = sqlalchemy.create_engine("postgresql://postgres:{}@34.145.47.78:5432/postgres".format(pwd))
    
    conn.raw_connection().set_session(autocommit=True)
    
    cur = conn.raw_connection().cursor()
    
except Exception as e:
    print("Error occurred while connecting to the database!")
    print(e)

In [162]:
names = pd.read_sql("stg_names", conn)
movies = pd.read_sql("stg_movies", conn)

In [88]:
movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,created_date
0,tt0138050,La vendetta dei morti viventi,Raiders of the Living Dead,1986,1986-03-01,Horror,86,USA,English,Samuel M. Sherman,...,A newspaper reporter hears of strange goings-o...,2.5,460,$ 800000,None,None,None,27.0,8.0,2022-01-05 01:28:04.053948
1,tt0138063,Un inguaribile romantico,So This Is Romance?,1997,1998-05-22,"Comedy, Romance",95,UK,English,Kevin W. Smith,...,Mike's alcoholic father tells him there is onl...,5.1,166,None,None,None,None,7.0,2.0,2022-01-05 01:28:04.053948
2,tt0138073,Os Saltimbancos Trapalhões,Os Saltimbancos Trapalhões,1981,1981-12-22,"Adventure, Comedy, Drama",95,Brazil,Portuguese,J.B. Tanko,...,Four humble and goofy employees of a prominent...,6.6,533,None,None,None,None,None,None,2022-01-05 01:28:04.053948
3,tt0138074,Santa and the Ice Cream Bunny,Santa and the Ice Cream Bunny,1972,1972-11-18,"Family, Fantasy",96,USA,English,Richard Winer,...,"When Santa's sleigh gets stuck in Florida, he ...",1.3,1186,None,None,None,None,51.0,24.0,2022-01-05 01:28:04.053948
4,tt0138075,Satan in High Heels,Satan in High Heels,1962,1967-08-03,Drama,90,USA,English,Jerald Intrator,...,A carnival burlesque dancer robs her junkie ex...,5.8,377,None,None,None,None,14.0,8.0,2022-01-05 01:28:04.053948


In [89]:
s = movies.isnull().sum() / len(movies) * 100
s.where(s>0).dropna().sort_values(ascending=False)

metascore                84.502941
usa_gross_income         82.148972
budget                   72.383670
worlwide_gross_income    63.873974
reviews_from_critics     13.740609
reviews_from_users        8.848640
production_company        5.188981
description               2.463456
writer                    1.830994
language                  0.970241
director                  0.101334
actors                    0.080368
country                   0.074544
dtype: float64

In [90]:
movies.duplicated(subset=["imdb_title_id"]).sum()

0

In [91]:
movies.dtypes

imdb_title_id                    object
title                            object
original_title                   object
year                             object
date_published                   object
genre                            object
duration                         object
country                          object
language                         object
director                         object
writer                           object
production_company               object
actors                           object
description                      object
avg_vote                         object
votes                            object
budget                           object
usa_gross_income                 object
worlwide_gross_income            object
metascore                        object
reviews_from_users               object
reviews_from_critics             object
created_date             datetime64[ns]
dtype: object

In [92]:
movies[["avg_vote", "votes", "budget", "usa_gross_income", "worlwide_gross_income", "metascore"]].dropna().sample(10)

,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore
69128,8.1,78200,RUR 1000000,$ 22168,$ 134788,90.0
68308,6.3,31416,$ 2500000,$ 12348905,$ 12348905,69.0
42043,7.3,9028,EUR 6000000,$ 1065665,$ 7400324,78.0
17039,6.4,37249,$ 40000000,$ 40946255,$ 103967384,71.0
37156,7.4,254949,$ 175000000,$ 364001123,$ 966552003,77.0
11291,6.5,238570,$ 60000000,$ 114197520,$ 256697520,44.0
48273,5.5,17278,$ 5000000,$ 3502600,$ 4554416,55.0
6893,6.6,60154,$ 25000000,$ 36845124,$ 64437847,58.0
73154,8.3,226427,DEM 32000000,$ 11487676,$ 11487676,86.0
78741,7.0,31038,$ 5300000,$ 2554476,$ 2554476,66.0


In [247]:
for col in movies:
    if movies[col].astype(str).str.contains(',').any():
        print(col)

title
original_title
genre
country
language
director
writer
production_company
actors
description


In [94]:
movies['imdb_title_id'].astype(str).str.contains(',', na=False).any()

False

In [95]:
print(movies.loc[movies['title'] != movies['original_title']].shape[0])

23372


In [96]:
movies[movies['title'] != movies['original_title']][['title', 'original_title']]

,title,original_title
0,La vendetta dei morti viventi,Raiders of the Living Dead
1,Un inguaribile romantico,So This Is Romance?
13,L'assassino ha prenotato la tua morte,Le temps de mourir
16,24 ore donna,The 24 Hour Woman
17,La trappola,Ambushed
...,...,...
85830,Arlington Road - L'inganno,Arlington Road
85834,Tre colpi che frantumano,Ma tou da jue dou
85835,Racconto d'autunno,Conte d'automne
85841,Relazione extraconiugale,Extramarital


In [97]:
movies[movies['date_published'].astype(str).str.len() != len("yyyy-mm-dd")]['date_published'].sample(10)

53848    1925
10741    1995
70992    1976
85349    1997
65949    1966
73080    1980
6538     1988
69063    1972
3280     1969
49451    2018
Name: date_published, dtype: object

In [98]:
movies['date_published'].isnull().sum()

0

In [99]:
movies[movies['year'] != movies['date_published'].astype(str).str.]

SyntaxError: invalid syntax (<ipython-input-99-fb77de13189d>, line 1)

In [100]:
date_check = pd.to_datetime(movies['date_published'])
date_check

0       1986-03-01
1       1998-05-22
2       1981-12-22
3       1972-11-18
4       1967-08-03
           ...    
85850   2000-01-20
85851   1966-08-26
85852   1954-01-01
85853   1988-02-01
85854   1970-01-01
Name: date_published, Length: 85855, dtype: datetime64[ns]

In [101]:
movies[date_check.dt.year != movies['year'].astype(int)][['year', 'date_published']]

,year,date_published
1,1997,1998-05-22
4,1962,1967-08-03
6,1998,1999-02-19
9,1998,1999-02-26
12,1971,1973-06-29
...,...,...
85834,1973,1976
85843,1997,1998-05-15
85849,1998,2005-01-05
85850,1999,2000-01-20


In [102]:
movies[["budget", "usa_gross_income", "worlwide_gross_income"]].dropna().sample(10)

,budget,usa_gross_income,worlwide_gross_income
19992,$ 1000000,$ 6870,$ 6870
45963,$ 12000000,$ 50722,$ 242115
28641,$ 1000000,$ 18464,$ 638274
7337,$ 56000000,$ 12882934,$ 12923936
17350,$ 30000000,$ 22537881,$ 27437881
74037,$ 17000000,$ 33000000,$ 33000000
74531,$ 8000000,$ 108423489,$ 108423489
4431,$ 75000000,$ 95011339,$ 248118121
15483,$ 130000000,$ 219964115,$ 459242249
28610,$ 5000000,$ 9206470,$ 10173061


In [103]:
non_usd_currency = movies[~movies['budget'].astype(str).str.startswith('$')]['budget'].dropna()

In [104]:
list_of_curr = non_usd_currency.astype(str).str.slice(0,3).unique().tolist()

In [105]:
len(list_of_curr)

78

In [106]:
non_usd_currency = movies[~movies['usa_gross_income'].astype(str).str.startswith('$')]['usa_gross_income'].dropna()
list_of_curr.extend(non_usd_currency.astype(str).str.slice(0,3).unique().tolist())

In [107]:
len(list_of_curr)

81

In [108]:
non_usd_currency = movies[~movies['worlwide_gross_income'].astype(str).str.startswith('$')]['worlwide_gross_income'].dropna()
list_of_curr.extend(non_usd_currency.astype(str).str.slice(0,3).unique().tolist())

In [109]:
len(list_of_curr)

85

In [110]:
set_of_curr = set(list_of_curr)
print(len(set_of_curr))

78


In [111]:
names.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children,created_date
0,nm0262745,Estelle Evans,Estelle Evans,None,"Estelle Evans was born on October 1, 1906 in R...","October 1, 1906 in Rolle Town, Bahamas",1906-10-01,"Rolle Town, Bahamas","July 20, 1985 in New York City, New York, USA ...",1985-07-20,"New York City, New York, USA",undisclosed,None,0,0,0,0,2022-01-05 01:29:06.999280
1,nm0262748,Evans Evans,Evans Evans,None,"Evans Evans was born on November 26, 1936 in B...","November 26, 1936 in Bluefield, West Virginia,...",1936-11-26,"Bluefield, West Virginia, USA",None,None,None,None,John Frankenheimer (13 December 1963 - 6 July...,1,0,0,0,2022-01-05 01:29:06.999280
2,nm0262753,Fiona Evans,Fiona Evans,None,"Fiona Evans is an actress, known for Unmasked ...",None,None,None,None,None,None,None,None,0,0,0,0,2022-01-05 01:29:06.999280
3,nm0262758,Frank Evans,Frank Evans,None,None,"September 3, 1899 in Kansas, USA",1899-09-03,"Kansas, USA","March 16, 1976 in Los Angeles, California, USA",1976-03-16,"Los Angeles, California, USA",None,None,0,0,0,0,2022-01-05 01:29:06.999280
4,nm0262762,François Evans,François Evans,165.0,"François Evans was born on August 7, 1965 in L...","August 7, 1965 in London, England, UK",1965-08-07,"London, England, UK",None,None,None,None,None,0,0,0,0,2022-01-05 01:29:06.999280


In [112]:
null_names_pct = names.isnull().sum() / len(names) * 100
null_names_pct.where(null_names_pct > 0).dropna().sort_values(ascending=False)

reason_of_death    92.377018
place_of_death     87.558825
death_details      86.586386
date_of_death      86.586386
height             84.991518
spouses_string     84.766128
place_of_birth     65.068776
birth_details      62.845098
date_of_birth      62.845098
bio                31.241330
dtype: float64

In [113]:
names.dtypes

imdb_name_id                     object
name                             object
birth_name                       object
height                           object
bio                              object
birth_details                    object
date_of_birth                    object
place_of_birth                   object
death_details                    object
date_of_death                    object
place_of_death                   object
reason_of_death                  object
spouses_string                   object
spouses                          object
divorces                         object
spouses_with_children            object
children                         object
created_date             datetime64[ns]
dtype: object

In [114]:
print(names.duplicated(subset=['imdb_name_id']).any())

False


In [115]:
for col in names: 
    if names[col].astype(str).str.contains(',').any():
        print(col)

name
birth_name
bio
birth_details
date_of_birth
place_of_birth
death_details
date_of_death
place_of_death
reason_of_death
spouses_string


In [116]:
names[names['date_of_birth'].astype(str).str.contains(',')][['place_of_birth','date_of_birth','place_of_death','date_of_death']].dropna().sample(20)

,place_of_birth,date_of_birth,place_of_death,date_of_death
13640,"Southwark, London, England, UK","1904 in Southwark, London, England, UK","Buckinghamshire, England, UK","1968 in Buckinghamshire, England, UK"
273966,"Riverstone, New South Wales, Australia","1906 in Riverstone, New South Wales, Australia","Los Angeles, California, USA",1972-07-10
34235,"London, England, UK","c. 1914 in London, England, UK","Auckland, New Zealand",1999-05-21
297079,"Newport News, Virginia, USA","1919 in Newport News, Virginia, USA","Atlanta, Georgia, USA","1983 in Atlanta, Georgia, USA (heart attack)"
272710,"Philadelphia, Pennsylvania, USA","1898 in Philadelphia, Pennsylvania, USA","Philadelphia, Pennsylvania, USA",1984-11-01
61212,"Grodno, Poland, Russian Empire [now Hrodna, Be...","1915 in Grodno, Poland, Russian Empire [now Hr...","Lódz, Lódzkie, Poland",1975-09-28
265702,"Tashkent, USSR","1924 in Tashkent, USSR","Tashkent, Uzbekistan",2015-06-12
84874,"Szeged, Hungary","1903 in Szeged, Hungary","Rome, Italy",1981-02-07
77759,"London, England, UK","1866 in London, England, UK","Los Angeles, California, USA",1920-03-21
282456,"Calcutta, Bengal Presidency, British India","1902 in Calcutta, Bengal Presidency, British I...","Calcutta, West Bengal, India","1980 in Calcutta, West Bengal, India"


In [189]:
names_from_comma = set()
for idx, val in movies[~movies['director'].str.title()
                      .isin([x.title().strip() for x in names['name'].to_list()])]['director'].str.split(',').items():
    try:
        names_list = [str.title().strip() for str in val]
        names_from_comma.update(names_list)
    except:
        pass

In [190]:
names_diff_dir = names_from_comma - set([s.title().strip() for s in names['name']])
print(len(names_diff))

1


In [193]:
print(names_diff_dir)

{'Ewelina Lukaszewska'}


In [194]:
name_list = names['name'].to_list()
writer_names = movies[~movies['writer'].str.title().isin([x.title().strip() for x in name_list])]['writer'].dropna()
writer_names

0                  Samuel M. Sherman, Brett Piper
2                     Renato Aragão, Luis Bacalov
4                 Harold Bonnett, John T. Chapman
5               Daniel Erickson, David Lane Smith
6                Nathan Long, Richard Preston Jr.
                           ...                   
85846                Seppo Huunonen, Lionel White
85847        Erkko Kivikoski, Marja-Leena Mikkola
85849             Frank Rehwaldt, George Saunders
85851                 Ere Kokkonen, Spede Pasanen
85852    Rajinder Singh Bedi, Saadat Hassan Manto
Name: writer, Length: 55242, dtype: object

In [195]:
writer_name_set = set()
for idx, writer in writer_names.str.split(',').items():
    names_list = [x.title().strip() for x in writer]
    writer_name_set.update(names_list)
    
print(len(writer_name_set))

51482


In [196]:
# writer_name_set = set()
# for writer in writer_names:
#     names_list = [x.title().strip() for x in writer.split(',')]
#     writer_name_set.update(names_list)
    
# print(writer_name_set)

In [197]:
name_diff_wir = writer_name_set - set([x.title().strip() for x in name_list])
print(len(name_diff))

0


In [200]:
name_diff = set() 
name_diff.update(names_diff_dir)
name_diff.update(name_diff_wir)
len(name_diff)

87

In [211]:
actor_names = movies[~movies['actors'].str.title().isin([x.title().strip() for x in name_list])]['actors'].dropna().str.strip().to_list()
type(actor_names)

list

In [212]:
actor_names_comma = set()
for actor in actor_names:
    actor_list = [x.title().strip() for x in actor.split(',')]
    actor_names_comma.update(actor_list)
    
len(actor_names_comma)

417223

In [214]:
name_diff_act = actor_names_comma - set([x.title().strip() for x in name_list])
print(len(name_diff_act))
name_diff.update(name_diff_act)
print(len(name_diff))

270445
270520


In [215]:
ratings = pd.read_sql("stg_ratings", conn)

In [216]:
ratings.head()

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,created_date
0,tt0028270,6.4,247,6.6,6.0,13,10,28,69,82,...,17.0,6.8,34.0,6.0,67.0,6.5,147.0,6.0,48.0,2022-01-05 01:31:04.793093
1,tt0028272,6.1,282,6.3,6.0,14,11,24,74,72,...,19.0,6.2,39.0,6.1,43.0,6.1,170.0,6.1,43.0,2022-01-05 01:31:04.793093
2,tt0028282,6.5,278,6.5,7.0,14,15,41,79,61,...,8.0,7.1,29.0,6.4,54.0,6.6,139.0,6.2,74.0,2022-01-05 01:31:04.793093
3,tt0028283,5.2,134,5.5,5.0,12,4,5,13,29,...,2.0,6.8,8.0,5.1,33.0,5.3,74.0,4.6,37.0,2022-01-05 01:31:04.793093
4,tt0028284,6.1,105,6.1,6.0,5,4,7,22,33,...,2.0,5.3,6.0,5.8,27.0,6.2,55.0,4.8,17.0,2022-01-05 01:31:04.793093


In [229]:
rating_pct = ratings.isnull().sum()/len(ratings) * 100 
rating_pct.where(rating_pct > 0).dropna().sort_values(ascending=False)

females_0age_votes           74.239124
females_0age_avg_vote        74.239124
males_0age_votes             68.072914
males_0age_avg_vote          68.072914
allgenders_0age_votes        61.144954
allgenders_0age_avg_vote     61.144954
females_18age_votes           7.595364
females_18age_avg_vote        7.595364
females_45age_votes           3.258983
females_45age_avg_vote        3.258983
males_18age_avg_vote          1.706365
males_18age_votes             1.706365
females_30age_votes           1.099528
females_30age_avg_vote        1.099528
allgenders_18age_votes        0.822317
allgenders_18age_avg_vote     0.822317
top1000_voters_votes          0.790868
top1000_voters_rating         0.790868
us_voters_votes               0.243434
us_voters_rating              0.243434
males_45age_votes             0.117640
males_45age_avg_vote          0.117640
females_allages_votes         0.094345
females_allages_avg_vote      0.094345
allgenders_45age_votes        0.093180
allgenders_45age_avg_vote

In [233]:
ratings.dtypes

imdb_title_id                        object
weighted_average_vote                object
total_votes                          object
mean_vote                            object
median_vote                          object
votes_10                             object
votes_9                              object
votes_8                              object
votes_7                              object
votes_6                              object
votes_5                              object
votes_4                              object
votes_3                              object
votes_2                              object
votes_1                              object
allgenders_0age_avg_vote             object
allgenders_0age_votes                object
allgenders_18age_avg_vote            object
allgenders_18age_votes               object
allgenders_30age_avg_vote            object
allgenders_30age_votes               object
allgenders_45age_avg_vote            object
allgenders_45age_votes          

In [234]:
for col in ratings:
    if ratings[col].astype(str).str.contains(',').any():
        print(col)

In [236]:
ratings.duplicated(subset=['imdb_title_id']).sum()

0

In [244]:
movies['country'].unique()

array(['USA', 'UK', 'Brazil', ..., 'Greece, Sweden',
       'Greenland, Denmark, Norway, Sweden',
       'France, Germany, Norway, Iceland'], dtype=object)